# Match most similar competency

## Existing competencies

In [54]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [55]:
competencys = ['Creativity', 'Employee loyalty', 'Reliability', 'Leadership', 'Passion', 'People Skills', 'Proactivity', 'Team Work']
for i in range(8):
    print "{}:{}".format(i, competencys[i])


0:Creativity
1:Employee loyalty
2:Reliability
3:Leadership
4:Passion
5:People Skills
6:Proactivity
7:Team Work


In [56]:
descriptions = ['Ability to think in a divergent way, and generate novel ideas.',
                'Be committed to the success of the organization and believe that working for this organization is their best option.',
                'Ability to performing to perform when needed, finishing projects and meeting deadlines.', 
                'Ability to engage and facilitate others to create a better result.', 
                'Desire to provide long-term commitment to their organization, demonstrate peak performance, and maintain increased tenure with the organization. With the intense desire or enthusiasm for the work.', 
                'Ability to establish social connections and understand the mindset of others.', 
                'Take the initiative and responsibility in improving business, rather than looking for causes in outside circumstances or other people.',
                'Ability to adapt to the needs of a group of people, while supporting the work of others.']

## load dictionary and corpus

In [57]:
from gensim import corpora, similarities, models

dictionary = corpora.Dictionary.load_from_text('/mnt/local/var/seedlink/data/dictionary/dict_en.txt')
print(dictionary)

Dictionary(50920 unique tokens: [u'sucess', u"cake'", u'mid-week', u'stallions', u'sonja']...)


In [58]:
import codecs
import json
path = '/mnt/local/var/seedlink/data/nltk_data/corpora/prepcorp/en_cc.json'
with codecs.open(path, 'r', 'utf8') as fp:
    jdata = fp.read()
data = json.loads(jdata)
corpus = data

In [59]:
for i in range(10):
    words =  [dictionary[t[0]] for t in corpus[i]]
     #print corpus[i]
    #print ' '.join(words)

## New competency

In [60]:
new_comp = """
Charisma	Able to establish social connections and a broad network
Confidence	Acts on his/her own initiative with confidence
Creativity	Seeks change and performs creatively
Entrepreneurial	Keeps aware of both organizational issues and market opportunities
Goal Orientated	Works systematically and drives the project to its goals
Goal Orientation	Work systematically and drive the project to the target goals
Growth Potential	Growth Potential
Integrity	Acts with integrity and shows social responsibility
Intellect	Flexible application of knowledge to differing contexts
Job Performance	Job Performance
Leadership	Identifies talents, empowers and motivates other
Logic	Logical thinking and rational analysis
Organisation	Sets objectives, balances resources and time, monitors progress
Organization	Sets objectives, balances resources and time, monitors progress
Performance	Job Performance
Proactive	Proactively deals with ambiguity
Process driven	Demonstrate commitment, accountablity, follow policies and procedures of the organizations
Resilient	Coping and resilient under stressful environment
Self motivated	Work enthusiastically for the achievement of self-development
Strategic Competency	Strategic Competency
Strategy	Foresight for potential problems and comes up with appropriate solutions
Teamwork	Adapts to the team; Supports others
"""

lines = new_comp.lower().strip().split('\n')
new_comp_dict = {splitted[0]:splitted[1] for splitted in [line.split('\t') for line in lines]}

## Tfidf

In [61]:
tfidf = models.TfidfModel(corpus)

In [62]:
desc_bow = [dictionary.doc2bow(desc.lower().split()) for desc in descriptions]
desc_tfidf = tfidf[desc_bow]

#index = similarities.MatrixSimilarity(desc_tfidf) # transform corpus to LSI space and index itb

In [63]:
# query by competency name
#for comp in new_comp_dict:
#    query = comp.lower()
##    query_bow = dictionary.doc2bow(query.split())
 #   query_tfidf = tfidf[query_bow]

#    sims = index[query_tfidf]
#    sims = sorted(enumerate(sims), key=lambda item: -item[1])
#    print "{:25}\t{:20}\t{}".format(comp, competencys[sims[0][0]], sims[0][1])

In [64]:
# query by competency desc
#for comp in new_comp_dict:
#    query = new_comp_dict[comp].lower()
#    query_bow = dictionary.doc2bow(query.split())
#    query_tfidf = tfidf[query_bow]

#    sims = index[query_tfidf]
#    sims = sorted(enumerate(sims), key=lambda item: -item[1])
#    print "{:25}\t{:20}\t{}".format(comp, competencys[sims[0][0]], sims[0][1])

## LSI

In [65]:
lsi = models.LsiModel(tfidf[corpus], id2word=dictionary, num_topics=100)

In [66]:
lsi.save('/tmp/comp-100.lsi')

In [67]:
lsi.add_documents(desc_tfidf)

In [68]:
index = similarities.MatrixSimilarity(lsi[desc_tfidf]) # transform corpus to LSI space and index it

In [69]:
# query by competency name
for comp in new_comp_dict:
    query = comp.lower()
    query_bow = dictionary.doc2bow(query.split())
    query_lsi = lsi[tfidf[query_bow]]

    sims = index[query_lsi]
    sims = sorted(enumerate(sims), key=lambda item: -item[1])
    print "{:25}\t{:20}\t{}".format(comp, competencys[sims[0][0]], sims[0][1])

strategic competency     	Reliability         	0.248677060008
growth potential         	Reliability         	0.229364946485
integrity                	Passion             	0.282940745354
intellect                	People Skills       	0.259692519903
confidence               	Passion             	0.342814087868
goal orientated          	Employee loyalty    	0.405059754848
job performance          	Team Work           	0.171646177769
organisation             	Reliability         	0.071084395051
creativity               	Proactivity         	0.0944112688303
self motivated           	Team Work           	0.289829999208
strategy                 	People Skills       	0.278630554676
charisma                 	Passion             	0.223079249263
teamwork                 	People Skills       	0.146099314094
resilient                	People Skills       	0.105503886938
performance              	Employee loyalty    	0.22753688693
goal orientation         	Employee loyalty    	0.424750328064
leadersh

In [70]:
# query by competency desc
for comp in new_comp_dict:
    query = new_comp_dict[comp].lower()
    query_bow = dictionary.doc2bow(query.split())
    query_lsi = lsi[tfidf[query_bow]]

    sims = index[query_lsi]
    sims = sorted(enumerate(sims), key=lambda item: -item[1])
    print "{:25}\t{:20}\t{}".format(comp, competencys[sims[0][0]], sims[0][1])

strategic competency     	Reliability         	0.248677060008
growth potential         	Reliability         	0.229364946485
integrity                	Passion             	0.601382493973
intellect                	People Skills       	0.859621524811
confidence               	Passion             	0.453828334808
goal orientated          	People Skills       	0.345883011818
job performance          	Team Work           	0.171646177769
organisation             	People Skills       	0.482191026211
creativity               	People Skills       	0.448025882244
self motivated           	Team Work           	0.47619509697
strategy                 	Passion             	0.504842936993
charisma                 	People Skills       	0.708268642426
teamwork                 	People Skills       	0.723624527454
resilient                	People Skills       	0.524786353111
performance              	Team Work           	0.171646177769
goal orientation         	Team Work           	0.70883077383
leadership